In [1]:

import numpy as np
import scipy as sp
import h5py
import os

import SimulatorData as sim_data
import Simulator as sim_system
import Optimization as opt


In [2]:

##! keys values are the columns of interest of exp data,
##! but the values are the ones used by the simulator 
##* the experimental data is read as it is presented, modfications are added later in the Simaulator object
##* gas species must be consistent with the chemical equations from the json file


schema = {
    "Pressure":     "pressure",
    "CurrentName":  "current",
    "WallTemp":     "Tw",
    "T near wall":  "Tnw", 
    "O mean":       "O",
    "N":            "N",
    "CO mean":      "CO",
    "gamma.1":      "gamma_exp"
}


###* Input files and function

reactions_file = "reactionsSimple.json"

const_dict = {
        "F0": 1.5e15,           # cm^-2
        "S0": 3e13,             # cm^-2
        
        "R": 0.00831442,        # kJ/mol*K
        "kBoltz": 1.380649e-23, # J/K
}

initial_state_dict = {'O_F': 0.1, 'O2_F':0.1 ,'O_S': 0.6, 'Vdb_S':0.1, 'Odb_S': 0.1}


###* Functions for the data transformation
def compute_flux(const_dict, exp_dict, specie, molar_mass):
    den = exp_dict.get(specie, 0.0)
    v_th = np.sqrt((8.0 * const_dict['R'] * 1000 * exp_dict['Tnw'])/(molar_mass * np.pi))
    flux = 0.25 * v_th * den * 100
    return flux


def compute_remaining_flux(const_dict, exp_dict, molar_mass): 
    den = exp_dict['N'] - exp_dict['O'] - exp_dict['CO']
    v_th = np.sqrt((8.0 * const_dict['R'] * 1000 * exp_dict['Tnw'])/(molar_mass * np.pi))
    flux = 0.25 * v_th * den * 100
    return flux

####? EavgMB data extracted from the Booth et al. 2019 paper
p_data_exp = [0.2, 0.3, 0.4, 0.5, 0.6, 0.75, 1.5]
EavgMB_data = [1.04, 0.91, 0.87, 0.83, 0.77, 0.5, 0.001]
interpolator = sp.interpolate.interp1d(p_data_exp, EavgMB_data, kind='linear', fill_value=0.001, bounds_error=False)


transformations_exp = {
    'Tw':       lambda const_dict, exp_dict: exp_dict['Tw'] + 273.15,
    'fluxO2' :  lambda const_dict, exp_dict: compute_remaining_flux(const_dict, exp_dict, 0.032),
    'fluxO' :   lambda const_dict, exp_dict: compute_flux(const_dict, exp_dict, 'O', 0.016),
    'fluxCO':   lambda const_dict, exp_dict: compute_flux(const_dict, exp_dict, 'CO', 0.028),
    'fluxCO2':  lambda const_dict, exp_dict: compute_flux(const_dict, exp_dict, 'CO2', 0.048),
    'EavgMB':   lambda const_dict, exp_dict: interpolator(exp_dict['pressure']).item(),
    'Ion':      lambda const_dict, exp_dict: 1e14 * exp_dict["current"]
}

input_folder_path = "Experimental_data_Paper"

output_folder_path = "Buffer_Data"
exp_data_file = input_folder_path + ".hdf5"

In [3]:

data_loader = sim_data.DataLoader(schema, folder_path=input_folder_path, output_file=exp_data_file, output_folder=output_folder_path)
data_loader.load_data(force_update=True)

Summary Buffer:
File Data : [b'Experimental_data_Paper/gammas_TD2019.xlsx']
col CO: shape(66,)
col File_Data: shape(1,)
col N: shape(66,)
col O: shape(66,)
col Tnw: shape(66,)
col Tw: shape(66,)
col current: shape(66,)
col gamma_exp: shape(66,)
col pressure: shape(66,)


In [4]:


exp_file = os.path.join(output_folder_path, exp_data_file)

sim = sim_system.Simulator(reactions_file, const_dict, exp_file, initial_state_dict, transformations_exp=transformations_exp)

print(sim.output_parser['rates_model'])
print(sim.output_parser['reactions_list'])




dO2_F/dt = -1.0*O2_F*O_F*r_15 - 1.0*O2_F*r_10 - 1.0*O2_F*r_12 - 1.0*O2_F*r_14 + 1.0*r_9*(-O2_F - O_F + 1.0)
dO_F/dt = -1.0*O2_F*O_F*r_15 - 2.0*O_F**2*r_8 - 0.02*O_F*O_S*r_7 - 0.02*O_F*Odb_S*r_27 - 0.02*O_F*Vdb_S*r_26 - 1.0*O_F*r_11 - 1.0*O_F*r_2 - 1.0*O_F*r_4 - 0.02*O_F*r_5*(-O_S - Odb_S - Vdb_S + 1.0) + 1.0*r_1*(-O2_F - O_F + 1.0)
dO_S/dt = -1.0*O_F*O_S*r_7 + 1.0*O_F*r_5*(-O_S - Odb_S - Vdb_S + 1.0) - 1.0*O_S*r_16 - 1.0*O_S*r_17 - 1.0*O_S*r_6 + 1.0*r_3*(-O_S - Odb_S - Vdb_S + 1.0)
dOdb_S/dt = -1.0*O_F*Odb_S*r_27 + 1.0*O_F*Vdb_S*r_26 - 1.0*Odb_S*r_23 - 1.0*Odb_S*r_24 - 1.0*Odb_S*r_25 + 1.0*Vdb_S*r_20
dVdb_S/dt = 1.0*O_F*Odb_S*r_27 - 1.0*O_F*Vdb_S*r_26 + 1.0*O_S*r_16 + 1.0*O_S*r_17 + 1.0*Odb_S*r_25 - 1.0*Vdb_S*r_20 - 1.0*Vdb_S*r_21 - 1.0*Vdb_S*r_22 + 1.0*r_18*(-O_S - Odb_S - Vdb_S + 1.0) + 1.0*r_19*(-O_S - Odb_S - Vdb_S + 1.0)
['r_1', 'r_2', 'r_3', 'r_4', 'r_5', 'r_6', 'r_7', 'r_8', 'r_9', 'r_10', 'r_11', 'r_12', 'r_14', 'r_15', 'r_16', 'r_17', 'r_18', 'r_19', 'r_20', 'r_21', 'r_22', 'r

In [5]:


def func_optimization(params):
    
    E, nu_D = params
    
    dict_mod_vec = [
    {"id": 1, "rate": "adsorption", "model_dict": {"E": E}},
    {"id": 8, "rate": None, "model_dict": {"nu_D": nu_D}}
    ]
    return dict_mod_vec

loss_function = lambda exp, teo: np.mean((np.reciprocal(exp)*(exp-teo))**2)

opt = opt.Optimizer(sim, func_optimization, loss_function)


In [6]:


frac_solutions_arr, gammas_result_arr, gamas_sum_arr = opt.solve_simulations((0.0, 1e13))


# print(frac_solutions_arr)
# print(gammas_result_arr)
print(gamas_sum_arr)


[0.00125796 0.0014356  0.00154653 0.0024537  0.00294202 0.00332242
 0.00154125 0.00168082 0.00172022 0.00218509 0.00282171 0.00332337
 0.00395099 0.00458441 0.00484624 0.00058163 0.00049913 0.00046387
 0.00053143 0.00068255 0.00080385 0.00097211 0.00110273 0.00079454
 0.00061466 0.00052725 0.00061035 0.00077691 0.00091746 0.00110741
 0.0013201  0.00145621 0.00045043 0.00034063 0.00027484 0.00029706
 0.00035499 0.00040116 0.00046218 0.00053802 0.0006347  0.00043166
 0.0003091  0.00032839 0.00039376 0.00045037 0.00054104 0.00067284
 0.00074725 0.00042904 0.00033168 0.0002702  0.00027877 0.00031038
 0.00033698 0.00037796 0.00043303 0.00059027 0.00041135 0.00029744
 0.00030642 0.00034671 0.00038439 0.00045262 0.00056855 0.00066593]
